# Download finBERT repo

In [1]:
! git clone https://github.com/ProsusAI/finBERT.git /finBERT
! env PYTHONPATH=/finBERT:$PYTHONPATH

Cloning into '/finBERT'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 168 (delta 6), reused 10 (delta 4), pack-reused 149
Receiving objects: 100% (168/168), 73.57 KiB | 353.00 KiB/s, done.
Resolving deltas: 100% (67/67), done.
SHELL=/bin/bash
NVIDIA_VISIBLE_DEVICES=all
CONDA_EXE=/root/miniforge3/bin/conda
_CE_M=
HOSTNAME=72fc73cfc941
PYTHONHASHSEED=0
XML_CATALOG_FILES=file:///root/miniforge3/envs/llm_sentiment/etc/xml/catalog file:///etc/xml/catalog
PWD=/code/notebooks
GSETTINGS_SCHEMA_DIR=/root/miniforge3/envs/llm_sentiment/share/glib-2.0/schemas
CONDA_ROOT=/root/miniforge3
CONDA_PREFIX=/root/miniforge3/envs/llm_sentiment
MAMBA_ROOT_PREFIX=/root/miniforge3
NVIDIA_DRIVER_CAPABILITIES=all
JPY_SESSION_NAME=/code/notebooks/3_finbert_benchmark.ipynb
GSETTINGS_SCHEMA_DIR_CONDA_BACKUP=
_=/usr/bin/env
HOME=/root
MAMBA_NO_BANNER=true
FORCE_COLOR=1
CONDA_PROMPT_MODIFIER=(llm_sentiment) 
PYD

# Download finBERT model

In [2]:
! git lfs install
! git clone https://huggingface.co/ProsusAI/finbert /code/models/ProsusAI_finbert

Git LFS initialized.
fatal: destination path '/code/models/ProsusAI_finbert' already exists and is not an empty directory.


# Import

In [21]:
from transformers import AutoModelForSequenceClassification
import argparse
import os
import sys
import pandas as pd
import nltk
from tqdm import tqdm

# turn off logging and futurewarnings
import logging 
logging.disable(logging.CRITICAL)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.insert(0,'/finBERT')
from finbert.finbert import predict

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Download data

In [4]:
ext_df = pd.read_csv('/code/data/extracted_data.csv')
ext_df.head()

,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16 21:00:00+00:00,Rolls-Royce can grow market share and improve ...,GE,https://www.nasdaq.com/articles/rolls-royce-ca...,NaN,NaN,"By Paul Sandle, Sarah Young and Tim Hepher\nLO...",But his challenge is to convince airlines afte...,Long-standing customer Thai Airways is finalis...,Based on underlying aircraft deliveries and ex...,Our engine will be very competitive to GE's en...
1,2023-12-16 00:00:00+00:00,General Electric's (GE) Unit Wins Wind Turbine...,GE,https://www.nasdaq.com/articles/general-electr...,NaN,NaN,"General Electric Company’s GE unit, GE Vernova...","Per the deal, GE will supply approximately 693...",Image Source: Zacks Investment Research Zacks ...,"General Electric Company’s GE unit, GE Vernova...","General Electric Company’s GE unit, GE Vernova..."
2,2023-12-16 00:00:00+00:00,FOCUS-North American aviation companies get la...,GE,https://www.nasdaq.com/articles/focus-north-am...,NaN,NaN,"By Allison Lampert\nMONTREAL, Dec 19 (Reuters)...","Business jet maker Bombardier BBDb.TO, which h...","Business jet maker Bombardier BBDb.TO, which h...",As clients like engine maker General Electric ...,As clients like engine maker General Electric ...
3,2023-12-16 00:00:00+00:00,Airbus on course for record jetliner orders in...,GE,https://www.nasdaq.com/articles/airbus-on-cour...,NaN,NaN,"By Tim Hepher\nPARIS, Dec 19 (Reuters) - Airbu...",Airlines are scrambling to order new planes to...,"Despite the positive end-year note, Airbus is ...","Despite the positive end-year note, Airbus is ...","With just over 10 days to go, the total has re..."
4,2023-12-15 00:00:00+00:00,Validea Detailed Fundamental Analysis - GE,GE,https://www.nasdaq.com/articles/validea-detail...,NaN,NaN,Below is Validea's guru fundamental report for...,Below is Validea's guru fundamental report for...,Detailed Analysis of GENERAL ELECTRIC CO GE Gu...,"Of the 22 guru strategies we follow, GE rates ...","Of the 22 guru strategies we follow, GE rates ..."


# Load the model

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('/code/models/ProsusAI_finbert', num_labels=3, cache_dir=None, device_map = 'cuda:1')

# Predict sentiment of article headlines

Run through the article headlines and predict the sentiments. For each row we save the date, ticker, url and the predicted sentiment to `/code/data/{filename}.csv` where the filename cohtains the ticker and the model used for the prediction.

In [23]:
goal_ticker = 'GE'
model_name = 'ProsusAI_finBERT'
filename = f'{goal_ticker}_{model_name}'

sentiments = []
get_token = lambda x: len(x.split(' '))/0.7

for idx, row in tqdm(ext_df.iterrows(), total=len(ext_df)):
    # remove sentence ending symbols, as finBERT runs on sentences and split texts to sentences to predict each
    headline = row['Article_title'].replace('.', '').replace('!', '').replace('?', '')
    
    sentiment = float((predict(headline, model, write_to_csv=False, use_gpu=True, batch_size=5)['sentiment_score']+1)*2.5)
    
    sentiments.append({'Stock_symbol': goal_ticker, 'Date': row.Date, 'Url': row.Url, 'Sentiment': sentiment})
    
    if (idx % 500) == 0:
        pd.DataFrame(sentiments).to_csv(f'/code/data/{filename}.csv', index=False)
    
pd.DataFrame(sentiments).to_csv(f'/code/data/{filename}.csv', index=False)

100%|██████████| 8680/8680 [01:45<00:00, 82.22it/s]
